In [1]:
import os, sys, subprocess, re, requests, json, warnings
import numpy as np
import pandas as pd
from urllib.request import urlopen
from time import sleep
from Bio import SeqIO

from Bio.PDB.DSSP import DSSP
from Bio.PDB.DSSP import PDBParser


warnings.filterwarnings('ignore')

In [10]:
import biotite.application.dssp as dssp

In [2]:
def get_url(url, **kwargs):
    '''
    Obatin a response from a given url
    '''
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [3]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb/"

# 1. FASTA sequence

In [4]:
# protein list import
df = pd.read_excel('./SourceData/protein_list.xlsx')
columns = df.iloc[0, 1:]
df = df.iloc[1:, 1:]
df.columns = columns

In [5]:
# Protein entry list
entry_list = df.Entry.unique().tolist()

# AH location start postion list
AH_start_list = df.AH_location_start.unique

In [6]:
# Obtain FASTA seq from Uniprot, make AH position data
for entry in df.Entry:
    print(entry)
    try:
        r = get_url(f'{WEBSITE_API}search?query=accession:{entry}&format=fasta&compressed=false')
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        break
        
    # export fasta
    fasta = r.text
    with open('./TrainingData/Original_fasta/%s.fas' % entry, 'w') as f:
        f.write(fasta)
        f.close()
        
    sleep(1)

Q9UH99
Q8N6T3
Q15643
P20606


# 2. Helix sequence extracted from AlphaFold predicted struture

## DSSP - primarily used

In [ ]:
file_path = '../../../AF-Q9WU40-F1-model_v3.pdb'

In [31]:
p = PDBParser()

In [70]:
def get_aaSeq_and_ss(pdb_filepath):

    # empty string for output    
    res_seq = ''
    ss_seq = ''
    
    # get dssp from the PDB file
    strcuture = p.get_structure('name', pdb_filepath)
    model = strcuture[0]
    dssp = DSSP(model, pdb_filepath)
    
    # parse dssp for residue symbol and ss info
    for i in range(len(dssp.keys())): # len of the protein
        # residue key
        a_key = list(dssp.keys())[i]

        # residue symbol and SS
        res = list(dssp[a_key])[1]
        ss = list(dssp[a_key])[2]
        res_seq += res
        ss_seq += ss

    return res_seq, ss_seq

In [71]:
res_seq, ss_seq = get_aaSeq_and_ss(file_path)

In [84]:
def find_Helix(res_seq, ss_seq):
    '''
    Return residues that belong to helix
    as a list of residues and their locations
    Note that '*' is added in the end of both AA and prediction strings,
    thus precition[0-1] and predition[last] return non-H anyway,
    preventing the bug when predciton starts or ends with H
    '''
    helix_res_loc_dict = dict()
    
    res_seq += '*'
    ss_seq += '*'

    for i in range(len(ss_seq)):
        if (ss_seq[i-1] != 'H') & (ss_seq[i] == 'H'):
            helix_start = i

        elif (ss_seq[i-1] == 'H') & (ss_seq[i] != 'H'):
            helix_end = i
            
            # pack helix residues and locations into a dictionary
            helix_res = res_seq[helix_start:helix_end]
            helix_loc = str(helix_start+1) + '-' + str(helix_end) # residue number is non-pythonic and starts with 1, not 0
            helix_res_loc_dict[helix_res] = helix_loc

    return helix_res_loc_dict

In [85]:
find_Helix(res_seq, ss_seq)

{'DEELFSQLRR': '15-24',
 'RPVYLKKLKKLREEEQQQQQQQQQQQHRA': '37-65',
 'EEELLQQFKRE': '461-471',
 'SFSAHYLSMFLLTAACLFFLILGLTYLGMR': '478-507',
 'ESEKNLLMSTLYKLHDRLAQIAGDHEC': '531-557',
 'VQEAAAYLKNL': '567-577',
 'EDVFNTSLLWIFKN': '582-595',
 'FWCRFRRAFITVTHRLLLLCLGVVLVCVALRYMRYRWTKEEEETRQMYDMVVKIIDVLRSHNEACQE': '630-696',
 'LPHVRDSL': '706-713',
 'KKVWDRAVDFLAAN': '723-736',
 'WHLAIQEAILEK': '812-823',
 'PEYAGKAFKAL': '851-861',
 'LDRYHHR': '878-884',
 'SHLR': '906-909'}

## Using biotite - but not used

In [8]:
# from tempfile import gettempdir, NamedTemporaryFile
# import biotite.structure as struc
# import biotite.structure.io as strucio
# import biotite.structure.io.pdb as pdb
# import biotite.database.rcsb as rcsb

# file_path = rcsb.fetch("1l2y", "pdb", gettempdir())
# pdb_file = pdb.PDBFile.read(file_path)
# tc5b = pdb_file.get_structure()
# print(type(tc5b).__name__)
# # print(tc5b.stack_depth())
# print(tc5b.array_length())
# print(tc5b.shape)

# array = strucio.load_structure(file_path)
# sse = struc.annotate_sse(array, chain_id='A')

# sse = ''.join(sse)

In [ ]:
dssp = DSSP(model, '/local-pdb/1mot.pdb', dssp='mkdssp')

## s4pred for ss prediction - unused

In [73]:
model = '../s4pred/run_model.py'
for entry in df.Entry:
    command = \
    'python %s --outfmt fas ./TrainingData/Original_fasta/%s.fas > ./TrainingData/s4pred_output/%s_ss.fas' \
    % (model, entry, entry)
    subprocess.run(command, shell=True)
    print(entry, ' done')

# file = open('./TrainingData/s4pred_output/Q15643_ss.fas', 'r')
# iterator = SeqIO.read(file, 'fasta')
# # records = SeqIO.to_dict(iterator)
# print(iterator.seq)
# # print(list)
# #     print(seq_rec.seq)
# file.close()


# # lines = file.readlines()
# # lines[1:]
# # seq = ''.join(lines[1:])

# DeepTMHMM

In [48]:
import biolib

In [52]:
deeptmhmm = biolib.load('DTU/DeepTMHMM')

# # Run DeepTMHMM
# deeptmhmm_res = deeptmhmm.cli(args='--fasta ./TrainingData/Original_fasta/Q15643.fas')

# # Save the results
# deeptmhmm_res.save_files("./biolib_results/")

# deeptmhmm_res.ipython_markdown()